In [3]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [5]:
# First let me read the order report file

order_report = pd.read_excel("Company X - Order Report.xlsx")

In [6]:
order_report # So this is the order_report data which we will play with.

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0
397,2001806226,8904223818850,2.0
398,2001806210,8904223816214,1.0


In [10]:
#let me create a data frame for this order_report data just for the sake of it.

df = order_report["ExternOrderNo"]

In [12]:
#Now lets read the next file i.e SKU_Master.

sku_master = pd.read_excel("Company X - SKU Master.xlsx")

sku_master.head() #Now that its read let me look at top 5 head quick.

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [14]:
#Cool now let me read the next file i.e. pincode_zones.

dfpincode_zones = pd.read_excel("Company X - Pincode Zones.xlsx")

dfpincode_zones #Let me look real quick.

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


In [15]:
# Onto the next lets read the invoice file

dfinvoice = pd.read_excel("Courier Company - Invoice.xlsx")

dfinvoice

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4
...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0


In [16]:
#Onto the next file rates

dfrates = pd.read_excel("Courier Company - Rates.xlsx")

dfrates

,Zone,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,A,0.50,29.5,23.6,13.6,23.6
1,B,1.00,33.0,28.3,20.5,28.3
2,C,1.25,40.1,38.9,31.9,38.9
3,D,1.50,45.4,44.8,41.3,44.8
4,E,2.00,56.6,55.5,50.7,55.5


In [35]:
#We will use merge to merge the dataframe of sku master and order report and then look how to find total weight.
# we are using merge and not join because we need to merge two column data frames and not index.

merge1df = sku_master.merge(order_report, how = "inner", on = "SKU") # This will merge em on basis of SKU numbers.

merge1df["Total weight(in kg)"] = merge1df["Weight (g)"] * merge1df["Order Qty"]/1000

#This will give us the total weight in kg. It will be for eg.> 0.25 kg or something since its in gm. 
# And we get it by multiplying it with how much orders were there or qty / 1000.

merge1df.drop_duplicates()                      #There are many sku with same number so this will drop duplicates.

merge1df = merge1df.merge(df).drop_duplicates() # This was imp to merge with df otherwise you won't get the merged table below

merge1df

#Okay so many things, firstly used [] brackets for calculations or I get errors so.
# Secondly I didn't group by because of an error probably a bug which says dataframegroup by has no attr. so ignore.

,SKU,Weight (g),ExternOrderNo,Order Qty,Total weight(in kg)
0,8904223815682,210,2001808801,1.0,0.210
4,8904223815859,165,2001808801,1.0,0.165
8,8904223815866,113,2001808801,2.0,0.226
12,8904223817273,65,2001808801,2.0,0.130
16,8904223815859,165,2001820978,1.0,0.165
...,...,...,...,...,...
1692,8904223819505,210,2001809917,1.0,0.210
1695,8904223819499,210,2001809917,1.0,0.210
1698,8904223819512,210,2001809917,1.0,0.210
1701,8904223819499,210,2001806885,2.0,0.420


In [39]:
#Okay now merging dfinvoice with merge1df and then cocatenate them with dfpincode_zones

merge2 = dfinvoice.merge(merge1df, how = "inner", left_on = "Order ID", right_on = "ExternOrderNo") # This will merge em.

#So order ID will be on left in the invoice

finaldf = pd.concat([merge2, dfpincode_zones], axis = 1) #concatenate it on columns basis

finaldf

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),SKU,Weight (g),ExternOrderNo,Order Qty,Total weight(in kg),Warehouse Pincode,Customer Pincode,Zone
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223818645,137,2001806232,6.0,0.822,121003.0,507101.0,d
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223819147,240,2001806232,2.0,0.480,121003.0,486886.0,d
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818706,127,2001806273,1.0,0.127,121003.0,532484.0,d
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818850,240,2001806273,1.0,0.240,121003.0,143001.0,b
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818942,133,2001806273,1.0,0.133,121003.0,515591.0,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,1091121844806,2001811475,0.5,121003,173212,b,Forward charges,33.0,8904223819499,210,2001811475,1.0,0.210,NaN,NaN,NaN
394,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223816214,120,2001811305,1.0,0.120,NaN,NaN,NaN
395,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223819505,210,2001811305,1.0,0.210,NaN,NaN,NaN
396,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223819499,210,2001811305,1.0,0.210,NaN,NaN,NaN


In [43]:
# so the weight slabs for each zone are given in df rates starting from 0.50 for a to 2.00 for zone e

# Lets start with weight slab by courier company.

def weightslabCC(wt):
    if wt <= 0.5:
        return 0.5
    
    elif wt > 0.5 and wt < 1:
        return 1
    
    elif wt > 1 and wt < 1.25:
        return 1.25
    
    elif wt > 1.25 and wt < 1.5:
        return 1.5
    
    elif wt > 1.5 and wt < 2:
        return 2
    
    else:
        return int(wt)+0.5
    

In [44]:
finaldf["Weight slab courier company"] = finaldf["Charged Weight"].apply(weightslabCC)

finaldf     #The above code applied weight slab to charged weights and is as per courier company.

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),SKU,Weight (g),ExternOrderNo,Order Qty,Total weight(in kg),Warehouse Pincode,Customer Pincode,Zone,Weight slab courier company
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223818645,137,2001806232,6.0,0.822,121003.0,507101.0,d,1.5
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223819147,240,2001806232,2.0,0.480,121003.0,486886.0,d,1.5
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818706,127,2001806273,1.0,0.127,121003.0,532484.0,d,1.5
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818850,240,2001806273,1.0,0.240,121003.0,143001.0,b,1.5
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818942,133,2001806273,1.0,0.133,121003.0,515591.0,d,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,1091121844806,2001811475,0.5,121003,173212,b,Forward charges,33.0,8904223819499,210,2001811475,1.0,0.210,NaN,NaN,NaN,0.5
394,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223816214,120,2001811305,1.0,0.120,NaN,NaN,NaN,0.5
395,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223819505,210,2001811305,1.0,0.210,NaN,NaN,NaN,0.5
396,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223819499,210,2001811305,1.0,0.210,NaN,NaN,NaN,0.5


In [45]:
# Now lets do Weight slab for company X
# Since we have already derived the condition by defining a function weightslabcc, we just need to use it again for total  wt.

finaldf[" Weight slab for company X "] = finaldf["Total weight(in kg)"].apply(weightslabCC)

finaldf

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),SKU,Weight (g),ExternOrderNo,Order Qty,Total weight(in kg),Warehouse Pincode,Customer Pincode,Zone,Weight slab courier company,Weight slab for company X
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223818645,137,2001806232,6.0,0.822,121003.0,507101.0,d,1.5,1.0
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223819147,240,2001806232,2.0,0.480,121003.0,486886.0,d,1.5,0.5
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818706,127,2001806273,1.0,0.127,121003.0,532484.0,d,1.5,0.5
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818850,240,2001806273,1.0,0.240,121003.0,143001.0,b,1.5,0.5
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818942,133,2001806273,1.0,0.133,121003.0,515591.0,d,1.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,1091121844806,2001811475,0.5,121003,173212,b,Forward charges,33.0,8904223819499,210,2001811475,1.0,0.210,NaN,NaN,NaN,0.5,0.5
394,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223816214,120,2001811305,1.0,0.120,NaN,NaN,NaN,0.5,0.5
395,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223819505,210,2001811305,1.0,0.210,NaN,NaN,NaN,0.5,0.5
396,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223819499,210,2001811305,1.0,0.210,NaN,NaN,NaN,0.5,0.5


In [62]:
# So basically the question of expected charges given in description is not clear enough to understand.
# Even the example is not clearly described in understandable language. The example on slabs and charges isn't clear too.
# So i will try on my own to calculate expected charges and then difference.

# But from here onward my calculations might be wrong, but i'll still try.


finaldf ['Expected charges x'] = finaldf ['Billing Amount (Rs.)']* finaldf ['Weight slab courier company'] / finaldf[" Weight slab for company X "]

finaldf # I was getting key error because the correct df is finaldf[" Weight slab for company X "] 
        # and i had not put proper spaces in between as it is so i got key error, corrected now.

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),SKU,Weight (g),ExternOrderNo,Order Qty,Total weight(in kg),Warehouse Pincode,Customer Pincode,Zone,Weight slab courier company,Weight slab for company X,Expected charges x
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223818645,137,2001806232,6.0,0.822,121003.0,507101.0,d,1.5,1.0,202.5
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223819147,240,2001806232,2.0,0.480,121003.0,486886.0,d,1.5,0.5,405.0
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818706,127,2001806273,1.0,0.127,121003.0,532484.0,d,1.5,0.5,270.6
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818850,240,2001806273,1.0,0.240,121003.0,143001.0,b,1.5,0.5,270.6
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818942,133,2001806273,1.0,0.133,121003.0,515591.0,d,1.5,0.5,270.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,1091121844806,2001811475,0.5,121003,173212,b,Forward charges,33.0,8904223819499,210,2001811475,1.0,0.210,NaN,NaN,NaN,0.5,0.5,33.0
394,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223816214,120,2001811305,1.0,0.120,NaN,NaN,NaN,0.5,0.5,45.4
395,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223819505,210,2001811305,1.0,0.210,NaN,NaN,NaN,0.5,0.5,45.4
396,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223819499,210,2001811305,1.0,0.210,NaN,NaN,NaN,0.5,0.5,45.4


In [75]:
# Difference between expected charges x company and billing amount i.e. courier company

# Charges expected by x are more somewhere so lets do it.

finaldf ['Difference between charges'] = finaldf['Expected charges x'] - finaldf['Billing Amount (Rs.)']

finaldf

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),SKU,Weight (g),ExternOrderNo,Order Qty,Total weight(in kg),Warehouse Pincode,Customer Pincode,Zone,Weight slab courier company,Weight slab for company X,Expected charges x,Difference between charges
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223818645,137,2001806232,6.0,0.822,121003.0,507101.0,d,1.5,1.0,202.5,67.5
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,8904223819147,240,2001806232,2.0,0.480,121003.0,486886.0,d,1.5,0.5,405.0,270.0
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818706,127,2001806273,1.0,0.127,121003.0,532484.0,d,1.5,0.5,270.6,180.4
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818850,240,2001806273,1.0,0.240,121003.0,143001.0,b,1.5,0.5,270.6,180.4
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,8904223818942,133,2001806273,1.0,0.133,121003.0,515591.0,d,1.5,0.5,270.6,180.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,1091121844806,2001811475,0.5,121003,173212,b,Forward charges,33.0,8904223819499,210,2001811475,1.0,0.210,NaN,NaN,NaN,0.5,0.5,33.0,0.0
394,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223816214,120,2001811305,1.0,0.120,NaN,NaN,NaN,0.5,0.5,45.4,0.0
395,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223819505,210,2001811305,1.0,0.210,NaN,NaN,NaN,0.5,0.5,45.4,0.0
396,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,8904223819499,210,2001811305,1.0,0.210,NaN,NaN,NaN,0.5,0.5,45.4,0.0


In [74]:
del finaldf['Difference between X charged'] # i bymistakely added extra column, deleted now so ignore.

In [77]:
finaldf['Difference between charges']

0       67.5
1      270.0
2      180.4
3      180.4
4      180.4
       ...  
393      0.0
394      0.0
395      0.0
396      0.0
397      0.0
Name: Difference between charges, Length: 398, dtype: float64

In [85]:
def summary(smry):
    if smry == 0:
        return "correctly charged"
    
    elif smry >0:
        return "overcharged"
    
    elif smry <0:
        return "undercharged"
    
    else : return "same price"

In [87]:
finaldf["summaryofcharge"] = finaldf['Difference between charges'].apply(summary)

In [88]:
finaldf["summaryofcharge"]

0            overcharged
1            overcharged
2            overcharged
3            overcharged
4            overcharged
             ...        
393    correctly charged
394    correctly charged
395    correctly charged
396    correctly charged
397    correctly charged
Name: summaryofcharge, Length: 398, dtype: object

In [198]:
print(finaldf["summaryofcharge"].value_counts()["correctly charged"])

50


In [200]:
print(finaldf["summaryofcharge"].value_counts()["overcharged"])

348


In [101]:
pd.set_option('display.max_rows', None)

finaldf

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),SKU,Weight (g),...,Order Qty,Total weight(in kg),Warehouse Pincode,Customer Pincode,Zone,Weight slab courier company,Weight slab for company X,Expected charges x,Difference between charges,summaryofcharge
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,8904223818645,137,...,6.0,0.822,121003.0,507101.0,d,1.50,1.00,202.500,67.500,overcharged
1,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,8904223819147,240,...,2.0,0.480,121003.0,486886.0,d,1.50,0.50,405.000,270.000,overcharged
2,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,8904223818706,127,...,1.0,0.127,121003.0,532484.0,d,1.50,0.50,270.600,180.400,overcharged
3,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,8904223818850,240,...,1.0,0.240,121003.0,143001.0,b,1.50,0.50,270.600,180.400,overcharged
4,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,8904223818942,133,...,1.0,0.133,121003.0,515591.0,d,1.50,0.50,270.600,180.400,overcharged
5,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,8904223819017,115,...,1.0,0.115,121003.0,326502.0,d,1.50,0.50,270.600,180.400,overcharged
6,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,8904223818874,100,...,1.0,0.100,121003.0,208019.0,b,2.50,0.50,1123.000,898.400,overcharged
7,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,8904223819024,112,...,8.0,0.896,121003.0,140301.0,b,2.50,1.00,561.500,336.900,overcharged
8,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,8904223819352,165,...,1.0,0.165,121003.0,396001.0,d,2.50,0.50,1123.000,898.400,overcharged
9,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,8904223819437,552,...,2.0,1.104,121003.0,711106.0,d,2.50,1.25,449.200,224.600,overcharged


In [210]:
print(finaldf['Difference between charges'].sum(axis = 0))

# amount overcharged


108924.94999999998


In [193]:
count1= finaldf[finaldf['Difference between charges']==0]['Expected charges x']
amt1 = count1.sum()

amt1 # This is the amount correctly charged.

2558.9000000000005

In [211]:
countcorrect = 50
countover = 348
countunder = 0
amt2 = 108924.94999999998
amt3 = 0                      # Did this way because was getting errors so easier.

In [212]:
dict1={'':['Total orders where X has been correctly charged','Total orders where X has been overcharged','Total orders where X has been undercharged'],
        'Count':[countcorrect,countover,countunder],
        'Amount(Rs.)':[amt1,amt2,amt3]}

dict1

{'': ['Total orders where X has been correctly charged',
  'Total orders where X has been overcharged',
  'Total orders where X has been undercharged'],
 'Count': [50, 348, 0],
 'Amount(Rs.)': [2558.9000000000005, 108924.94999999998, 0]}

In [213]:
table1 = pd.DataFrame(dict1,index=[0,1,2])    # Putting the above data in table format by  giving proper indexing

table1

,,Count,Amount(Rs.)
0,Total orders where X has been correctly charged,50,2558.90
1,Total orders where X has been overcharged,348,108924.95
2,Total orders where X has been undercharged,0,0.00


In [215]:
# THE FINAL SUMMARY IS GIVEN ABOVE. ASSIGNMENT FOR COINTAB COMPLETE.

In [217]:
# OUTPUT TO AN EXCEL FILE

finaldf.to_excel('order level calculation all tables.xlsx',index=False)

print('file saved')

file saved


In [218]:
table1.to_excel('summary of charges in table.xlsx',index=False)

print('file saved successfully')

file saved successfully
